# Lesson 4 - Clustering

1. Read the Bernoulli Mixture Model Derivation.
2. Read about Stochastic Expectation-Maximization (EM) Algorithm: https://www.sciencedirect.com/science/article/pii/S0167947320302504.
3. From the given code, modify the EM algorithm to become a Stochastic EM Algorithm.
4. Use the data from the paper: https://www.sciencedirect.com/science/article/abs/pii/S0031320322001753
5. Perform categorical clustering using the Bernoulli Mixture Model with Stochastic EM Algorithm.
6. Compare its performance with K-Modes Algorithm using Folkes-Mallows Index, Adjusted Rand Index, and Normalized Mutual Information Score.
7. Compare and contrast the performances, and explain what is happening (i.e. why is FMI always higher than ARI and NMI? Why is ARI and NMI low compared to FMI? etc.)
8. Write the report in Latex, push to your github with the codes.

### Given Code

In [130]:
from scipy.special import logsumexp

class BernoulliMixtureEM:
    
    def __init__(self, n_components, max_iter, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.tol = tol
    
    def fit(self,x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            # E-Step
            self.gamma = self.get_responsibilities(log_bernoullis)
            self.remember_params()
            # M-Step
            self.get_Neff()
            self.get_mu()
            self.get_pi()
            # Compute new log_likelihood:
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if np.isnan(self.logL):
                self.reset_params()
                print(self.logL)
                break

    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)
        
    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()
    
    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()
    
    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]
            
    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma
        
    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis
    
    def get_save_single(self, x, mu):
        mu_place = np.where(np.max(mu, axis=0) <= 1e-15, 1e-15, mu)
        return np.tensordot(x, np.log(mu_place), (1,1))
        
    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)
    
    def get_mu(self):
        self.mu = np.einsum('ik,id -> kd', self.gamma, self.x) / self.Neff[:,None] 
        
    def get_pi(self):
        self.pi = self.Neff / self.n_samples
    
    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)
        
    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
    
    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))
        
    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)
    
    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

### Modified BernouliMixture

In [131]:
import numpy as np
from scipy.special import logsumexp

class StochasticBernoulliMixtureEM:
    def __init__(self, n_components, max_iter, batch_size=100, tol=1e-3):
        self.n_components = n_components
        self.max_iter = max_iter
        self.batch_size = batch_size
        self.tol = tol
        
    def fit(self, x):
        self.x = x
        self.init_params()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.old_logL = self.get_log_likelihood(log_bernoullis)
        for step in range(self.max_iter):
            if step > 0:
                self.old_logL = self.logL
            self.gamma = self.get_responsibilities(log_bernoullis)
            self.remember_params()
            self.get_Neff()
            self.get_mu(self.x)
            self.get_pi()
            log_bernoullis = self.get_log_bernoullis(self.x)
            self.logL = self.get_log_likelihood(log_bernoullis)
            if np.isnan(self.logL):
                self.reset_params()
                print(self.logL)
                break

    def iterate_batches(self):
        n_samples = len(self.x)
        indices = np.arange(n_samples)
        np.random.shuffle(indices)
        for start_idx in range(0, n_samples, self.batch_size):
            end_idx = min(start_idx + self.batch_size, n_samples)
            batch_indices = indices[start_idx:end_idx]
            yield self.x.iloc[batch_indices]

    def reset_params(self):
        self.mu = self.old_mu.copy()
        self.pi = self.old_pi.copy()
        self.gamma = self.old_gamma.copy()
        self.get_Neff()
        log_bernoullis = self.get_log_bernoullis(self.x)
        self.logL = self.get_log_likelihood(log_bernoullis)
        
    def remember_params(self):
        self.old_mu = self.mu.copy()
        self.old_pi = self.pi.copy()
        self.old_gamma = self.gamma.copy()
    
    def init_params(self):
        self.n_samples = self.x.shape[0]
        self.n_features = self.x.shape[1]
        self.pi = 1/self.n_components * np.ones(self.n_components)
        self.mu = np.random.RandomState(seed=0).uniform(low=0.25, high=0.75, size=(self.n_components, self.n_features))
        self.normalize_mu()
        self.old_mu = None
        self.old_pi = None
        self.old_gamma = None
    
    def normalize_mu(self):
        sum_over_features = np.sum(self.mu, axis=1)
        for k in range(self.n_components):
            self.mu[k,:] /= sum_over_features[k]
            
    def get_responsibilities(self, log_bernoullis):
        gamma = np.zeros(shape=(log_bernoullis.shape[0], self.n_components))
        Z =  logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
        for k in range(self.n_components):
            gamma[:, k] = np.exp(np.log(self.pi[k]) + log_bernoullis[:,k] - Z)
        return gamma
        
    def get_log_bernoullis(self, x):
        log_bernoullis = self.get_save_single(x, self.mu)
        log_bernoullis += self.get_save_single(1-x, 1-self.mu)
        return log_bernoullis
    
    def get_save_single(self, x, mu):
        epsilon = 1e-15
        mu_place = np.clip(mu, epsilon, 1 - epsilon)
        return np.tensordot(x, np.log(mu_place), (1,1))

    def get_Neff(self):
        self.Neff = np.sum(self.gamma, axis=0)
    
    def get_mu(self, batch):
        self.mu = np.einsum('ik,id -> kd', self.gamma, batch) / self.Neff[:, None]
        
    def get_pi(self):
        self.pi = self.Neff / self.n_samples
    
    def predict(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        gamma = self.get_responsibilities(log_bernoullis)
        return np.argmax(gamma, axis=1)
        
    def get_sample_log_likelihood(self, log_bernoullis):
        return logsumexp(np.log(self.pi[None,:]) + log_bernoullis, axis=1)
    
    def get_log_likelihood(self, log_bernoullis):
        return np.mean(self.get_sample_log_likelihood(log_bernoullis))
        
    def score(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_log_likelihood(log_bernoullis)
    
    def score_samples(self, x):
        log_bernoullis = self.get_log_bernoullis(x)
        return self.get_sample_log_likelihood(log_bernoullis)

### Fetch Data

In [4]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd

soybean= fetch_ucirepo(id=91) 
zoo = fetch_ucirepo(id=111) 
heart_disease = fetch_ucirepo(id=45) 
dermatology = fetch_ucirepo(id=33)
breast_cancer = fetch_ucirepo(id=15)
mushroom = fetch_ucirepo(id=73)

### Format Dataframes

In [132]:
import pandas as pd

datasets = [soybean, zoo, heart_disease, dermatology, breast_cancer, mushroom]
dataset_names = ['soybean', 'zoo', 'heart_disease', 'dermatology', 'breast_cancer', 'mushroom']
dfs = []

for dataset, name in zip(datasets, dataset_names):
    X = dataset.data.features
    y = dataset.data.targets
    df = pd.merge(X, y, left_index=True, right_index=True)
    df = df.dropna()
    dfs.append((name, df))

soybean_df, zoo_df, heart_disease_df, dermatology_df, breast_cancer_df, mushroom_df = dfs

### Categorical Clustering & Evaluation


In [134]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import fowlkes_mallows_score, adjusted_rand_score, normalized_mutual_info_score

def evaluate_clustering(true_labels, pred_labels):
    fmi = fowlkes_mallows_score(true_labels, pred_labels)
    ari = adjusted_rand_score(true_labels, pred_labels)
    nmi = normalized_mutual_info_score(true_labels, pred_labels)
    return fmi, ari, nmi
    
# Encode categorical features
def encode_categorical(df):
    label_encoders = {}
    for column in df.columns:
        if df[column].dtype == 'object':
            label_encoders[column] = LabelEncoder()
            df[column] = label_encoders[column].fit_transform(df[column])
    return df, label_encoders

# Apply label encoding to datasets
encoded_dfs = []
label_encoders_list = []
for name, df in dfs:
    encoded_df, label_encoders = encode_categorical(df)
    encoded_dfs.append((name, encoded_df))
    label_encoders_list.append(label_encoders)

# Apply clustering algorithms and evaluate performance
for name, df in encoded_dfs:
    print("Dataset:", name)
    if 'target' in df.columns:
        y_true = df['target'].values
        X = df.drop(columns=['target'])
    else:
        y_true = df.iloc[:, -1].values
        X = df.iloc[:, :-1]

    # Bernoulli Mixture Model with Stochastic EM Algorithm
    bm_model = StochasticBernoulliMixtureEM(n_components=3, max_iter=100)
    bm_model.fit(X)
    bm_clusters = bm_model.predict(X)

    # K-Modes Algorithm
    km_model = KModes(n_clusters=3, init='Huang', n_init=5, verbose=0)
    km_clusters = km_model.fit_predict(X)

    # Evaluate clustering performance
    bm_fmi, bm_ari, bm_nmi = evaluate_clustering(y_true, bm_clusters)
    km_fmi, km_ari, km_nmi = evaluate_clustering(y_true, km_clusters)

    # Print evaluation results
    print("Bernoulli Mixture Model:")
    print("Folkes-Mallows Index:", bm_fmi)
    print("Adjusted Rand Index:", bm_ari)
    print("Normalized Mutual Information Score:", bm_nmi)
    print("\nK-Modes Algorithm:")
    print("Folkes-Mallows Index:", km_fmi)
    print("Adjusted Rand Index:", km_ari)
    print("Normalized Mutual Information Score:", km_nmi)
    print("\n")


Dataset: soybean
Bernoulli Mixture Model:
Folkes-Mallows Index: 0.6518126479666801
Adjusted Rand Index: 0.5010966211000518
Normalized Mutual Information Score: 0.6887777884924488

K-Modes Algorithm:
Folkes-Mallows Index: 0.7839084587216347
Adjusted Rand Index: 0.653688872137944
Normalized Mutual Information Score: 0.8376655260685574


Dataset: zoo
Bernoulli Mixture Model:
Folkes-Mallows Index: 0.7226653463342378
Adjusted Rand Index: 0.5973920064427068
Normalized Mutual Information Score: 0.6342850921689538

K-Modes Algorithm:
Folkes-Mallows Index: 0.8158447903777043
Adjusted Rand Index: 0.727582121144255
Normalized Mutual Information Score: 0.7295757614209041


Dataset: heart_disease
Bernoulli Mixture Model:
Folkes-Mallows Index: 0.4373549066876981
Adjusted Rand Index: 0.14113304815085476
Normalized Mutual Information Score: 0.14791364947221453

K-Modes Algorithm:
Folkes-Mallows Index: 0.44812355734517517
Adjusted Rand Index: 0.1623242410982316
Normalized Mutual Information Score: 0.19

### Conclusion

When comparing the clustering performances of the Bernoulli Mixture Model with Stochastic EM Algorithm and the K-Modes Algorithm, it's evident that the Folkes-Mallows Index (FMI) consistently yields higher values compared to the Adjusted Rand Index (ARI) and Normalized Mutual Information Score (NMI). FMI provides a comprehensive assessment by considering both precision and recall of the clusters, indicating strong clustering performance. Conversely, ARI and NMI tend to show lower values compared to FMI. ARI measures the similarity between two clusterings while accounting for chance agreement, and its lower values imply some discrepancies in cluster assignments between the algorithms. Similarly, NMI measures the agreement between true labels and predicted clusters, and its lower values suggest that while clusters may have high precision and recall, they might not fully capture the same information as the true labels.

These disparities in performance metrics reflect the unique focus and sensitivities of each metric, as well as the characteristics of the algorithms and datasets. FMI's emphasis on both precision and recall provides a holistic evaluation, while ARI and NMI focus on different aspects of clustering agreement and mutual information, respectively. Factors such as the stochastic nature of the Bernoulli Mixture Model and the handling of categorical data by K-Modes Algorithm further contribute to variations in performance. Understanding these nuances aids in comprehensively assessing the strengths and limitations of each clustering algorithm in effectively capturing the underlying structure of the data.





